## Capstone project.

Notebook for 'Applied Data Science Capstone' project of IBM Data Science Professional Certificate.

In [2]:
# Installing map rendering library
!pip install folium

print('Library imported.')

     |████████████████████████████████| 102kB 5.7MB/s ta 0:00:011
Library imported.


In [3]:
# Installing location querying library (Postal Code)
!pip install pgeocode

print('Library imported.')

Library imported.


In [4]:
# Installing location querying library (City name)
!pip install geopy

print('Library imported.')

Library imported.


In [5]:
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# library for web scrapping
from bs4 import BeautifulSoup
import requests

# library for getting location coordinates 
import pgeocode # get lat, long values from postal code
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# library for map rendering
import folium 

# library to handle JSON files
import json 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# library for k-means from clustering stage
from sklearn.cluster import KMeans

## 1. Scrapping the table of Boroughs and Neighborhoods in Toronto from Wikipedia

For this task we will use BeautifulSoup package

In [6]:
# Getting the Wikipedia table site address 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
# Scrapping the site with BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"effb7f1c-76f8-4e4a-96e6-cb5115a410a2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [8]:
# Selecting all tables from scrapped data
soup_tables = soup.find_all('table')
soup_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [9]:
# Selecting the right table from all tables
right_table = soup.find('table',{'class':'wikitable sortable'})
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [10]:
# Extracting items from right table for each column
postal_codes = []
boroughs =[]
neighborhood =[]

for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        postal_codes.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))

In [11]:
# Testing for the results
myDict = {'Postal Code': postal_codes,
         'Borough': boroughs,
         'Neighborhood': neighborhood}
print(myDict)

{'Postal Code': ['M1A\n', 'M2A\n', 'M3A\n', 'M4A\n', 'M5A\n', 'M6A\n', 'M7A\n', 'M8A\n', 'M9A\n', 'M1B\n', 'M2B\n', 'M3B\n', 'M4B\n', 'M5B\n', 'M6B\n', 'M7B\n', 'M8B\n', 'M9B\n', 'M1C\n', 'M2C\n', 'M3C\n', 'M4C\n', 'M5C\n', 'M6C\n', 'M7C\n', 'M8C\n', 'M9C\n', 'M1E\n', 'M2E\n', 'M3E\n', 'M4E\n', 'M5E\n', 'M6E\n', 'M7E\n', 'M8E\n', 'M9E\n', 'M1G\n', 'M2G\n', 'M3G\n', 'M4G\n', 'M5G\n', 'M6G\n', 'M7G\n', 'M8G\n', 'M9G\n', 'M1H\n', 'M2H\n', 'M3H\n', 'M4H\n', 'M5H\n', 'M6H\n', 'M7H\n', 'M8H\n', 'M9H\n', 'M1J\n', 'M2J\n', 'M3J\n', 'M4J\n', 'M5J\n', 'M6J\n', 'M7J\n', 'M8J\n', 'M9J\n', 'M1K\n', 'M2K\n', 'M3K\n', 'M4K\n', 'M5K\n', 'M6K\n', 'M7K\n', 'M8K\n', 'M9K\n', 'M1L\n', 'M2L\n', 'M3L\n', 'M4L\n', 'M5L\n', 'M6L\n', 'M7L\n', 'M8L\n', 'M9L\n', 'M1M\n', 'M2M\n', 'M3M\n', 'M4M\n', 'M5M\n', 'M6M\n', 'M7M\n', 'M8M\n', 'M9M\n', 'M1N\n', 'M2N\n', 'M3N\n', 'M4N\n', 'M5N\n', 'M6N\n', 'M7N\n', 'M8N\n', 'M9N\n', 'M1P\n', 'M2P\n', 'M3P\n', 'M4P\n', 'M5P\n', 'M6P\n', 'M7P\n', 'M8P\n', 'M9P\n', 'M1R\n', 'M

### Creating the Dataframe

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [12]:
# Create the dataframe and assign each list into a column
df = pd.DataFrame(postal_codes, columns=['PostalCode'])
df['Borough'] = boroughs
df['Neighborhood'] = neighborhood
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
# Checking the created DataFrame
df.describe()

,PostalCode,Borough,Neighborhood
count,180,180,180
unique,180,11,100
top,M1H,Not assigned,Not assigned
freq,1,77,77


In [14]:
# Removing the "\n" from values
df = df.replace('\n','', regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [15]:
# Check how many rows have "Not assigned" in the Borough column
df[(df['Borough'] == "Not assigned")].count()

PostalCode      77
Borough         77
Neighborhood    77
dtype: int64

In [16]:
# Exclude rows where Borough = "Not assigned"
condition = df[ (df['Borough'] == 'Not assigned')].index
df.drop(condition, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
# Check for the result 
print(df[(df['Borough'] == "Not assigned")].count())
print(df[(df['Borough'] != "Not assigned")].count())

# 103 remained + 77 rows excluded (line 120) = 180 original rows (line 118)

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64
PostalCode      103
Borough         103
Neighborhood    103
dtype: int64


- More than one neighborhood can exist in one postal code area. \
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. \
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [18]:
# Check for repetitions in PostalCode
any(df['PostalCode'].duplicated())

False

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [19]:
# Check how many rows have "Not assigned" in the Neighborhood column
df[(df['Neighborhood'] == "Not assigned")].count()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

In [20]:
# Resetting the index
df.reset_index(inplace=True, drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [21]:
df.shape

(103, 3)

## 2. Getting the latitude and the longitude coordinates of each neighborhood

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.\
We will use the Pgeocode Python package 

In [22]:
# Define "country = Canada" for request
nomi = pgeocode.Nominatim('ca')  

# Create the list of Postal Codes 
a = df[['PostalCode']]

# loop request for each postal code
for i in enumerate(a):
  g = nomi.query_postal_code(a.astype(str).values)

In [23]:
# Create DataFrame with results
df1 = pd.DataFrame(g)
df1.head()

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,M3A,CA,North York (York Heights / Victoria Village / ...,Ontario,ON,North York,NaN,NaN,NaN,43.7545,-79.3300,1.0
1,M4A,CA,North York (Sweeney Park / Wigmore Park),Ontario,ON,NaN,NaN,NaN,NaN,43.7276,-79.3148,6.0
2,M5A,CA,Downtown Toronto (Regent Park / Port of Toronto),Ontario,ON,Toronto,8133394.0,NaN,NaN,43.6555,-79.3626,6.0
3,M6A,CA,North York (Lawrence Manor / Lawrence Heights),Ontario,ON,North York,NaN,NaN,NaN,43.7223,-79.4504,6.0
4,M7A,CA,Queen's Park Ontario Provincial Government,Ontario,ON,NaN,NaN,NaN,NaN,43.6641,-79.3889,NaN


In [24]:
# select desirable columns
df1 = df1[['postal_code', 'latitude', 'longitude']]

# Rename column names
df1 = df1.rename(columns={'postal_code':'PostalCode', 'latitude':'Latitude', 'longitude':'Longitude'})

df1.head()

,PostalCode,Latitude,Longitude
0,M3A,43.7545,-79.3300
1,M4A,43.7276,-79.3148
2,M5A,43.6555,-79.3626
3,M6A,43.7223,-79.4504
4,M7A,43.6641,-79.3889


### Merging coordinates data into the original dataframe

In [25]:
# Merging both DataFrames on the same postal code
df_coords = pd.merge(df, df1, on='PostalCode')
df_coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [26]:
# Checking for missing data
df_coords.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        1
Longitude       1
dtype: int64

In [27]:
# Delete row with missing data
df_coords.dropna(axis=0, inplace=True)

df_coords.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [28]:
df_coords.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


## 3. Generating a map of  the neighborhoods with clustered venues

### Creating a map of Toronto with selected neighborhoods superimposed on top

- Selecting neighborhoods that have "Toronto" as part of their names  

In [29]:
# Creating Dataframe with constraint rules
toronto_data = df_coords[df_coords['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


- Using Geopy and Folium to create the map

In [30]:
#Using Geolocator to get Toronto's coordinates 
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [31]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Extracting data from Foursquare API

In [32]:
# Define Foursquare Credentials and Version
CLIENT_ID = '3XXXU3NWENQFPDTUR11XXJJYKDILP2ZQCFG2NYUCK2HCZHN0' # your Foursquare ID
CLIENT_SECRET = 'NZPEZCDBC5F4NXKFTWFHDYVT5BVO3EHBY0DQ2O0LTONESQHZ' # your Foursquare Secret
VERSION = '20200710' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3XXXU3NWENQFPDTUR11XXJJYKDILP2ZQCFG2NYUCK2HCZHN0
CLIENT_SECRET:NZPEZCDBC5F4NXKFTWFHDYVT5BVO3EHBY0DQ2O0LTONESQHZ


- Example of Forsquare API request to get the top 500 venues that are within a radius of 1000 meters from Toronto

In [33]:
# Create the get request
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3XXXU3NWENQFPDTUR11XXJJYKDILP2ZQCFG2NYUCK2HCZHN0&client_secret=NZPEZCDBC5F4NXKFTWFHDYVT5BVO3EHBY0DQ2O0LTONESQHZ&v=20200710&ll=43.6534817,-79.3839347&radius=1000&limit=500'

- Create a function to extract the top 100 venues that are within a radius of 500 meters from each neighborhood 

In [34]:
# Function that returns relevant information for each nearby venue
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',                   'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
# run the function on each neighborhood and create a new dataframe
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'],
                                 latitudes = toronto_data['Latitude'],
                                 longitudes = toronto_data['Longitude']
                                 )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [36]:
# Let's check the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1538, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


- Let's check how many venues were returned for each neighborhood

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,93,93,93,93,93,93
"Brockton, Parkdale Village, Exhibition Place",38,38,38,38,38,38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",59,59,59,59,59,59
Central Bay Street,64,64,64,64,64,64
Christie,12,12,12,12,12,12
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


- Let's find out how many unique categories can be curated from all the returned venues

In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


### Analyzing each neighborhood

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# Checking the size of the DataFrame
toronto_onehot.shape

(1538, 213)

- Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.010753,0.000000,0.010753,0.021505,0.000000,0.000000,0.000000,0.000000,0.010753,...,0.000000,0.000000,0.00,0.010753,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.026316,0.000000,0.026316,0.000000,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.016949,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.015625,0.015625,0.00,0.015625,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Church and Wellesley,0.028169,0.000000,0.014085,0.000000,0.000000,0.014085,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [42]:
# Confirm the new size
toronto_grouped.shape

(39, 213)

- Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1               Hotel  0.05
2                Café  0.05
3              Bakery  0.04
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2  Thrift / Vintage Store  0.05
3          Breakfast Spot  0.05
4               Gift Shop  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0     Restaurant  0.19
1    Coffee Shop  0.12
2    Yoga Studio  0.06
3           Bank  0.06
4  Deli / Bodega  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3                 Bar  0.05
4   French Restaurant  0.03


----Central Bay Street----
                  

### Let's put that information into a pandas dataframe

In [44]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gift Shop,Breakfast Spot,Thrift / Vintage Store,Italian Restaurant,Restaurant,Gym,Mexican Restaurant,Boutique
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Bank,Martial Arts Dojo,Breakfast Spot,Bookstore,Furniture / Home Store,Sushi Restaurant,Japanese Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,Bar,Speakeasy,French Restaurant,Park,Gym / Fitness Center,Bank,Bakery
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Clothing Store,Italian Restaurant,Pizza Place,Café,Gastropub,Donut Shop


### Clustering Neighborhoods
- Let's run k-means to cluster the neighborhood into 5 clusters

In [46]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 4], dtype=int32)

- Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Coffee Shop,Gym,Park,Creperie,Diner,Distribution Center,College Theater,Ramen Restaurant,Beer Bar,Mexican Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Hotel,Japanese Restaurant,Italian Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Bookstore,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Department Store
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Trail,Bakery,Cheese Shop,Gastropub,Health Food Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dog Run


- Finally, let's visualize the map with the resulting clusters

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4 . Analyze clusters

- Cluster 1, Label = 0

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,Downtown Toronto,0,Coffee Shop,Gym,Park,Creperie,Diner,Distribution Center,College Theater,Ramen Restaurant,Beer Bar,Mexican Restaurant
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Hotel,Japanese Restaurant,Italian Restaurant,Café,Cosmetics Shop,Bubble Tea Shop,Bookstore,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Department Store
4,East Toronto,0,Pub,Trail,Bakery,Cheese Shop,Gastropub,Health Food Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dog Run
5,Downtown Toronto,0,Coffee Shop,Hotel,Café,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Farmers Market
6,Downtown Toronto,0,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Clothing Store,Italian Restaurant,Pizza Place,Café,Gastropub,Donut Shop
8,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Gym,Hotel,Thai Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant
11,West Toronto,0,Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Cocktail Bar,Coffee Shop,Vietnamese Restaurant,Restaurant,Record Shop,Deli / Bodega,Pizza Place
12,East Toronto,0,Greek Restaurant,Ice Cream Shop,Café,Restaurant,Italian Restaurant,Yoga Studio,Liquor Store,Cocktail Bar,Coffee Shop,Bubble Tea Shop


- Cluster 2, Label = 1

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Clothing Store,Home Service,Women's Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


- Cluster 3, Label = 2

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Photography Studio,Park,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


- Cluster 4, Label = 3

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Gym Pool,Playground,Park,Garden,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
33,Downtown Toronto,3,Playground,Park,Grocery Store,Candy Store,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


- Cluster 5, Label = 4

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,4,Café,Grocery Store,Park,Coffee Shop,Playground,Candy Store,Athletics & Sports,Baby Store,Fast Food Restaurant,Farmers Market
9,West Toronto,4,Park,Grocery Store,Bakery,Pizza Place,Bus Line,Smoke Shop,Middle Eastern Restaurant,Furniture / Home Store,Bar,Bank
10,Downtown Toronto,4,Harbor / Marina,Park,Café,Music Venue,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
20,Central Toronto,4,Food & Drink Shop,Dog Run,Gym / Fitness Center,Park,Hotel,Breakfast Spot,Department Store,Falafel Restaurant,Electronics Store,Ethiopian Restaurant
21,Central Toronto,4,Cosmetics Shop,Park,Bus Line,Trail,Doner Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
22,West Toronto,4,Park,Residential Building (Apartment / Condo),Dive Bar,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
29,Central Toronto,4,Tennis Court,Park,Gym,Grocery Store,Thai Restaurant,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
